In [32]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import random
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm
import numdifftools as nd #Using this package to numerically approximate the hessian matrix
import time #Used to measure time to run code
from scipy.stats import norm

In [51]:
start_time = time.time()
folder='Insert your path here! xD)'
data=np.genfromtxt(folder+'SP500_from2012.csv', delimiter=',') #loading in dataset
x = data[1:, 4:5] #100 times log-returns of the S&P 500 index from 2012
x=x.T[0,:] #unpacking numpy array
T=len(x)
omega = 0.497556 #MLE of omega 
a = 0.208533 #//MLE of alpha
alpha = 0.05
loss=np.zeros(T-2)
loss=-(x[1:T-1] + x[2:T]) #//future two-period loss
M=10000 #number of monte carlo simulations
temp1=np.zeros(M)
temp2=-np.sqrt(np.var(x))*np.sqrt(2)*norm.ppf(alpha) #two period VaR for iid Gaussian returns

VaR=np.zeros(T-2) #VaR variable
VaR_Gauss=np.zeros(T-2) #VaR gaussian variable

for i in range(T-2):
    for j in range(M):
        z=np.random.normal(loc=0.0, scale=1.0, size=2) #draw innovations from N(0,1)
        r1=np.sqrt(omega+a*x[i]**2)*z[0] #//compute return at time i+1
        r2=np.sqrt(omega+a*r1**2)*z[1] #//compute return at time i+2
        temp1[j]=-(r1+r2)#//compute two-period loss
    VaR[i]=np.quantile(temp1,1-alpha) #// compute (1-alpha) percentile of losses
    VaR_Gauss[i]=temp2 #VaR under Gaussianity
print('Elapsed time (all computations)+'"--- %s seconds ---" % (time.time() - start_time))

Elapsed time (all computations)+--- 87.05097508430481 seconds ---


In [56]:
#exporting to CSV file
df=pd.DataFrame(loss, columns=["2_period_loss"])
df["2_period_VaR_Gauss"]=VaR_Gauss
df["2_period_VaR_ARCH"]=VaR

my_folder='Insert your path here! xD)'
#my_folder='C:/Users/vjule/OneDrive/Dokumenter/skole/Kandidatfag/Financial_econometrics/Undervisning/Koder_ders_skal_konverteres/'
df.to_csv(my_folder+"simVaR.csv")